In [21]:
import numpy as np

In [22]:
#random test problem

d = 10
l = 100
data1x = np.random.randint(255, size=(d-1,l))
data1y = np.random.randint(255, size=(1,l))
data2x = np.random.randint(255, size =(d-1,l))
data2y = -np.random.randint(-20,255, size=(1,l))
data1 = np.concatenate((data1x,data1y))
data2 = np.concatenate((data2x,data2y))
data = np.concatenate((data1,data2), axis = 1)
data = np.transpose(data)

label1 = np.ones(l).astype(int)
label2 = - label1;
label = np.concatenate((label1,label2))

#print(label)



### Maximal violating pair SMO

In [23]:
def F(i,alpha):
    
    out = - label[i]
    for j in range(l):
        out += alpha[j]*label[j]*kernel(data[i],data[j])
    return out

In [24]:
def I_membership_no(a,y,C):
    if 0 < a < C:
        return 0
    elif a == 0 and y == 1:
        return 1
    elif a == C and y == -1:
        return 2
    elif a == C and y == 1:
        return 3
    else:
        return 4

In [25]:
def smo_original(data,label,C,kernel,tol,violationcheckyesorno):
    
    #bisher unnoetig
    #eps = 1e-12
    
    l = label.shape[0]
    alpha = np.zeros(l)
    
    fcache = -label.astype(float)
    
    b_up = [-1]
    b_low = [1]
    
    I = np.zeros((5,l), dtype=np.int)
    I[1,:] = (label == 1).astype(int)
    I[4,:] = (label == -1).astype(int)

    i_up = [min(np.nonzero(I[1,:])[0])]
    i_low = [min(np.nonzero(I[4,:])[0])]
    
    
    ##############################################################################################
    # auxiliary functions for takeStep and examineExample
    
    def I_membership(a,y):
        if 0 < a and a < C:
            return np.array([1,0,0,0,0])
        elif a == 0 and y == 1:
            return np.array([0,1,0,0,0])
        elif a == C and y == -1:
            return np.array([0,0,1,0,0])
        elif a == C and y == 1:
            return np.array([0,0,0,1,0])
        else:
            return np.array([0,0,0,0,1])
    

    def I_membership_no(a,y):
        if 0 < a < C:
            return 0
        elif a == 0 and y == 1:
            return 1
        elif a == C and y == -1:
            return 2
        elif a == C and y == 1:
            return 3
        else:
            return 4
        
    def F(i,alpha):
        
        #avoid for loop in case of standard scalar product kernel
        if kernel_identifier == 'standard scalar product':
            k = alpha * label;
            m = np.dot(data[:,i],data)
            out = -label[i] + np.dot(k,m)
        else: 
            out = - label[i]
            for j in range(l):
                out += alpha[j]*label[j]*kernel(data[:,i],data[:,j])
        return out
    
    
    ##############################################################################################   
    
    
    
    
    ##############################################################################################
    # subprocedure takeStep
    
    def takeStep(i1,i2):

    
        if i1 == i2: 
            return 0
        alph1 = alpha[i1]
        alph2 = alpha[i2]
        y1 = label[i1]
        y2 = label[i2]  
   
        F1 = fcache[i1]
        F2 = fcache[i2]
        s = y1*y2
    
        if s == -1:
            L = max(0,alph2-alph1)
            H = min(C,C+alph2-alph1)
        else:
            L = max(0,alph2+alph1-C)
            H = min(C,alph2+alph1)
    
        if L == H: return 0
    
        k11 = kernel(data[:,i1],data[:,i1])
        k12 = kernel(data[:,i1],data[:,i2])
        k22 = kernel(data[:,i2],data[:,i2])
    
        eta = 2*k12 - k11 - k22
        if eta < 0:
            a2 = alph2 - y2*(F1-F2)/eta
            if a2 < L:
                a2 = L
            elif a2 > H:
                a2 = H
        else:
            #print('Error: eta == 0')
            raise ValueError('Error: eta == 0')
    
        #if abs(a2-alph2) < eps*(a2+alph2+eps):
            #return 0
    
        a1 = alph1 + s*(alph2-a2)
    
   
        # update alpha
        alpha[i1] = a1
        alpha[i2] = a2
    
        # update I
        I[:,i1] =  I_membership(a1,y1)
        I[:,i2] =  I_membership(a2,y2)
    
        # update fcache[i] here for i in I_0: a[0:3][:,4:9]
        ind_0 = (np.nonzero(I[0,:]))[0]
        
        #avoid for loop in case of standard scalar product kernel
        if kernel_identifier == 'standard scalar product':
            fcache[ind_0] += y1*(a1-alph1)*np.dot(data[:,i1],data[:,ind_0]) + y2*(a2-alph2)*np.dot(data[:,i2],data[:,ind_0])
        else:
            for i in ind_0:
                fcache[i] = fcache[i] + y1*(a1-alph1)*kernel(data[:,i],data[:,i1]) + y2*(a2-alph2)*kernel(data[:,i],data[:,i2])
    
        # update fcache for indices i1 and i2
        fcache[i1] = F1 + y1*(a1-alph1)*k11 + y2*(a2-alph2)*k12
        fcache[i2] = F2 + y1*(a1-alph1)*k12 + y2*(a2-alph2)*k22
        
        
        b_low[0] = -float('inf')
        b_up[0] = - b_low[0]
        
        if np.size(ind_0) > 0:
            for i in ind_0:
                if fcache[i] > b_low[0]:
                    b_low[0] = fcache[i]
                    i_low[0] = i
                if fcache[i] < b_up:
                    b_up[0] = fcache[i]
                    i_up[0] = i
       
        memb1 = I_membership_no(a1,y1)
        memb2 = I_membership_no(a2,y2)
        if memb1 in (1,2) and fcache[i1] < b_up[0]:
            b_up[0] = fcache[i1]
            i_up[0] = i1
        if memb1 in (3,4) and fcache[i1] > b_low[0]:
            b_low[0] = fcache[i1]
            i_low[0] = i1
        if memb2 in (1,2) and fcache[i2] < b_up[0]:
            b_up[0] = fcache[i2]
            i_up[0] = i2
        if memb2 in (3,4) and fcache[i2] > b_low[0]:
            b_low[0] = fcache[i2]
            i_low[0] = i2

        return 1
    
    ##############################################################################################
    
    
    
    
    ##############################################################################################
    #subprocedure examineExample
    
    def examineExample(i2):

    
        y2 = label[i2]
        alph2 = alpha[i2]
        memb_i2 = I_membership_no(alph2,y2);
    
        # look up or compute F2:= F_i2
        if memb_i2 == 0:
            F2 = fcache[i2]
        else:
            fcache[i2] = F(i2,alpha)
            F2 = fcache[i2]
        
            # in case i2 not in I_0, see if F2 changes or estimate of b_low or b_up
            if memb_i2 in (1,2) and F2 < b_up[0]:
                b_up[0] = F2
                i_up[0] = i2
            elif memb_i2 in (3,4) and F2 > b_low[0]:
                b_low[0] = F2
                i_low[0] = i2
    
        optimality = 1
        if memb_i2 in (0,1,2):
            if b_low[0] - F2 > tol:
                optimality = 0
                i1 = i_low[0]
        if memb_i2 in (0,3,4):
            if F2 - b_up[0] > tol:
                optimality = 0
                i1 = i_up[0]
        if optimality == 1:
            return 0
    
        if memb_i2 == 0:
            if b_low[0] - F2 > F2 - b_up[0]:
                i1 = i_low[0]
            else:
                i1 = i_up[0]
            
        if takeStep(i1,i2):
            return 1
        else:
            return 0
    
    ##############################################################################################
    
    
    
    
    ##############################################################################################
    #main routine
    
    numChanged = 0
    examineAll = 1

    while numChanged > 0 or examineAll:
        numChanged = 0
        if examineAll:
            for i in range(l):
                numChanged = numChanged + examineExample(i)
        else:
            for i in (np.nonzero(I[0,:]))[0]:
                numChanged = numChanged + examineExample(i)
                if b_up[0] > b_low[0] - tol:
                    numChanged = 0
                    break
        if examineAll == 1:
            examineAll = 0
        elif numChanged == 0:
            examineAll = 1
            
            
    if violationcheckyesorno == 'yes':
        memb = [I_membership_no(alpha[i],label[i]) for i in range(l)]
        I_up = [memb[i] in (0,1,2) for i in range(l)]
        I_low = [memb[i] in (0,3,4) for i in range(l)]
        bs = [F(i,alpha) for i in range(l)]

        b_up = float('inf')
        b_low = -b_up
        for i in range(l):
            if I_up[i] == 1 and bs[i] < b_up:
                b_up = bs[i]
            if I_low[i] == 1 and bs[i] > b_low:
                b_low = bs[i]

        if b_low - tol <= b_up:
            violationstring = 'no tol-violation'
        else:
            violationstring = 'tol-violations!!'
    else:
        violationstring = 'no violation requested'

    
    return {'solution': alpha, 'violationcheck': violationstring}

    ##############################################################################################

In [26]:
def smo_new(data,label,C,kernel,tol,violationcheckyesorno):
    
    #sehr wichtig!
    null = 1e-16
    
    def I_up_low_membership(alpha_i,label_i):
        v = np.array([False,False])
        if (alpha_i < C-null and label_i == 1) or (alpha_i > null and label_i == -1):
            v[0] = True
        if (alpha_i < C-null and label_i == -1) or (alpha_i > null and label_i == 1):
            v[1] = True
        return v
        
        
    #data: images are rows of array, so the number of columns is 28**2 and the number of rows is the number of training data
    l = label.shape[0]
    alpha = np.zeros(l)
    
    I = np.zeros((2,l),dtype = bool)
                  
    #initialize I, now logical vector! I[0] = I_up, I[1] = I_low
    I[0] = label == 1
    I[1] = label == -1
    
        
    b_up = -1
    b_low = 1
    
    v = np.array(range(l))
    i_up_array = v[I[0]]
    i_low_array = v[I[1]]
    
    i_0 = i_up_array[0]
    j_0 = i_low_array[0]
    
    #hier stimmt fcache noch!!!!!!!!!!
    fcache = -label.astype(float)
    #print('fcache error: ', fcache - [F(i,alpha) for i in range(l)])
    
    iter = 0
    cycle = 0
    
    #kostenintensiv bei SMO mit maximal violating pairs ist das ständige Neuberechnen der kompletten Kernel-Matrix; solange also
    #initialisiere daher lxl - Nullmatrix (numpy macht sie sparse, das geht also) und speichere alle bisher berechneten kernel-Berechnung ab
    #die Idee ist, dass das funktionieren sollte, da die meisten alpha_i auf Null bleiben; also sollte auch unsere Gram-Matrix
    #sparse bleiben
    
    #sparse, i.e., no memory overflow so far
    K = np.zeros([l,l])
    
    rows_calc = []
    
    while(b_up < b_low - tol):

        alph1 = alpha[i_0]
        alph2 = alpha[j_0]
        
        y1 = label[i_0]
        y2 = label[j_0]
        
        F1 = fcache[i_0]
        F2 = fcache[j_0]
        
        s = y1*y2
                
        if s == -1:
            L = max(0,alph2-alph1)
            H = min(C,C+alph2-alph1)
        else:
            L = max(0,alph2+alph1-C)
            H = min(C,alph2+alph1)
        
        if i_0 not in rows_calc:
            K[i_0] = [kernel(data[i],data[i_0]) for i in range(l)]
            rows_calc.append(i_0)
        
        if j_0 not in rows_calc:
            K[j_0] = [kernel(data[i],data[j_0]) for i in range(l)]
            rows_calc.append(j_0)
            
        #print(K)
            
        k11 = K[i_0,i_0]
        k12 = K[i_0,j_0]
        k22 = K[j_0,j_0]
        
        eta = 2*k12 - k11 - k22
        
        if eta < 0:
            a2 = alph2 - y2*(F1-F2)/eta
            if a2 < L:
                a2 = L
            elif a2 > H:
                a2 = H
        else:
            #print('Error: eta == 0')
            raise ValueError('Error: eta == 0')
        
        a1 = alph1 + s*(alph2-a2)
        
        #zum Vergleich, siehe unten, wo untersucht wird, wie oft es vorkommt, dass
        #ein Paar nach einem Schritt sofort wieder violating ist
        alpha_old = np.empty(alpha.shape)
        np.copyto(alpha_old, alpha) 
        
        # 3.) update alpha, I_up, I_low and then fcache
        alpha[i_0] = a1
        alpha[j_0] = a2
        
                 
        I[:,i_0] = I_up_low_membership(a1,y1)
        I[:,j_0] = I_up_low_membership(a2,y2)
        
        #prüfe, ob I richtig geupdatet wurde: erfolgreich, I wird korrekt geupdatet

        #w = list(v)
        #w.remove(i_0)
        #w.remove(j_0)
        #print('alpha außerhalb von i_0,j_0 gleich wenn Null:',np.linalg.norm((alpha_old-alpha)[w]))  
        
        #fac_i_0 = y1 * (a1 - alph1)
        #fac_j_0 = y2 * (a2 - alph2)
        
        #fcache += fac_i_0 * K[i_0]  + fac_j_0 * K[j_0]
        
        
        #behelfsmäßig
        #fcache = np.array([F(i,alpha) for i in range(l)])
        #print('fcache error: ', fcache2 - fcache)
        
        # 4.) berechne neues i_0 und j_0  für maximally violating pair und dazu b_up, b_low
        
        
        #now choose i_0,j_0 for next iteration and compute b_up, b_low for these i_0,j_0
     
                
        #Alternative:
        
        I_up = I[0]
        I_low = I[1]
        
        i_0_old = i_0
        j_0_old = j_0
        b_up = float('inf')
        b_low = -b_up
        
        iupset = []
        ilowset = []
        for i in range(l):
            if I_up[i] == 1 and fcache[i] < b_up:
                iupset.append(i)
                b_up = fcache[i]
                i_0 = i
            if I_low[i] == 1 and fcache[i] > b_low:
                ilowset.append(i)
                b_low = fcache[i]
                j_0 = i
                
        #if i_0 == i_0_old:
            #i_0 = np.random.choice(iupset)
        #if j_0 == j_0_old:
            #j_0 = np.random.choice(ilowset)
        
        
        
        
        
        
        if (1):
            #print('alpha =', alpha)
            #print('b_up =', b_up)
            #print('b_low =', b_low)
            #print('b_up - b_low =', b_up - b_low)
            if i_0_old == i_0 and j_0_old == j_0:
                cycle += 1
                print('Achtung, Paar zweimal hintereinander violating:')
                print('i_0 =',i_0,'j_0 =',j_0)
                print('alpha[i_0] =',alpha[i_0], 'alpha[j_0] =',alpha[j_0])
                print('alpha[i_0]_old =',alph1, 'alpha[j_0]_old =',alph2)
                print('fcache[i_0] =',fcache[i_0], 'fcache[j_0] =', fcache[j_0])
                print('fcache[i_0]_old =',F(i_0,alpha_old), 'fcache[j_0]_old =', F(j_0,alpha_old))
                print('b_up - b_low =', b_up - b_low)
            #print('min(fcache[I[0]]) =', np.min(fcache[I[0]]))
            #print('max(fcache[I[1]]) =', np.max(fcache[I[1]]))
                  
            #print('alpha * label =', np.dot(alpha,label))
            #print('i_0 =', i_0 )
            #print('j_0 =', j_0)
            #print('min(alpha) =', np.min(alpha))
            #print('max(alpha) =', np.max(alpha))
           
            #print('fcache error2: ', fcache2 - [F(i,alpha) for i in range(l)])
            
        iter += 1
        
        
    if violationcheckyesorno == 'yes':
        
        I_up = I[0]
        I_low = I[1]
        #bs = fcache
        bs = np.array([F(i,alpha) for i in range(l)])

        b_up = float('inf')
        b_low = -b_up
        for i in range(l):
            if I_up[i] == 1 and bs[i] < b_up:
                b_up = bs[i]
            if I_low[i] == 1 and bs[i] > b_low:
                b_low = bs[i]

        if b_low - tol <= b_up:
            violationstring = 'no tol-violation'
        else:
            violationstring = 'tol-violations!!'
    else:
        violationstring = 'no violation requested'    
   
    print('Anzahl wiederholt dasselbe Paar =', cycle)
    print('iter =', iter)
    return {'solution': alpha, 'violationcheck': violationstring}, b_up, b_low

In [27]:
def kernel(v1,v2):
    """standard scalar product"""
    return np.dot(v1,v2)

kernel_identifier = kernel.__doc__

In [28]:
%%time
result = smo_new(data,label,1,kernel,1e-3,'yes')
print(result)

KeyboardInterrupt: 

In [ ]:
fcache = np.array([F(i,result[0]['solution']) for i in range(l)])

In [ ]:
%%time
result = smo_original(np.transpose(data),label,1,kernel,1e-3,'yes')
print(result)